<a href="https://colab.research.google.com/github/m-mehabadi/grad-maker/blob/main/notebooks/GradMaker_AllMethods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Method 1 Version 0

In [ ]:
def gradient_maker(domain_grads, epsilon=0.5, alpha=0.01):

    dgr = domain_grads
    number_of_domains, dim = dgr.shape
    g = np.zeros(dim)
    u_ = np.zeros(number_of_domains)

    # dgr_bar = np.sum(dgr, axis=1)/dim
    iter = 0
    while not np.min(dgr@g)/dim >= 0.99 * epsilon:
        if iter >= 100:
            break
        
        # print("||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||")
        # print(f"u is {u_}")
        # print(f"g is {g}")
        g_bar = np.sum(g)/dim
        # E = g_bar.item()*dim*dgr_bar
        inner = dgr@g
        print(f"Iter={iter}: g_bar={g_bar}, Inner=({inner[0]}, {inner[1]}), Condition={np.min(dgr@g)/dim}")


        u_ = u_ + alpha*(epsilon - (dgr@g)/dim)
        # g = (1./number_of_domains)*np.sum(((1+(u_>=0)*u_).reshape(number_of_domains, 1))*dgr, axis=0)
        g = (1./number_of_domains)*np.sum(((1+u_).reshape(number_of_domains, 1))*dgr, axis=0)
        iter += 1
    return g

### Method 1 Version 1: With standardization

In [ ]:
def gradient_maker(domain_grads, epsilon=0.5, alpha=0.01):
    # def log():
    #     # print("||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||")
    #     # print(f"u is {u_}")
    #     # print(f"g is {g}")
    #     g_bar = np.sum(g)/dim
    #     # E = g_bar.item()*dim*dgr_bar
    #     inner = dgr@g/dim
    #     print(f"Iter={iter}: g_bar={g_bar}, Inner=({inner[0]}, {inner[1]}), Condition={np.min(dgr@g/dim)}")

    dgr = domain_grads
    number_of_domains, dim = dgr.shape

    #
    _mean = np.mean(dgr, axis=1)[:, np.newaxis]
    _std = np.std(dgr, axis=1)[:, np.newaxis]
    dgr = (dgr-_mean)/_std
    
    #
    g = np.random.randn(dim)
    u_ = np.zeros(number_of_domains)
    
    iter = 0
    while not np.min(dgr@g/dim) >= epsilon:
        if iter >= 100:
            break
        
        # log()

        u_ = u_ + alpha*(1.01*epsilon - (dgr@g)/dim)
        # g = (1./number_of_domains)*np.sum(((1+(u_>=0)*u_).reshape(number_of_domains, 1))*dgr, axis=0)
        g = (1./number_of_domains)*np.sum(((1+u_).reshape(number_of_domains, 1))*dgr, axis=0)
        iter += 1
    # log()
    return g

### Method 1 Version 2: Normalization and scaling back

In [ ]:
def gradient_maker(domain_grads, epsilon=0.5, alpha=0.01, eff=0.1):
    ###### NOTES:
    ### make sure that `domain_grads` dtype is float precision not integer or long.

    def log():
        # print(f"g={g}")
        print(f"Iter={iter}, Condition={np.min(dgr@g/dim)}")
    
    def null(A, eps=1e-15):
        u, s, vh = np.linalg.svd(A)
        # print(vh)
        null_space = np.compress(s <= eps, vh, axis=0)
        return null_space.T

    def null(A):
        from scipy.linalg import null_space
        n = null_space(A, rcond=1)
        if n.shape[1] < 1:
            return False, None
        r = np.sqrt(np.sum((A@n)**2, axis=0)/n.shape[1])
        if np.min(r) >= 1e-2:
            return False, None
        return True, n.T[np.argmin(r)].reshape(-1, 1)
    
    def scale():
        G = np.concatenate((g.reshape(1, -1), domain_grads), axis=0)
        is_fine, n = null(G@G.T)
        if not is_fine:
            return np.mean(np.sqrt(np.sum(domain_grads**2, axis=1)))/np.sqrt(np.sum(g**2))
        # print(n)
        n /= np.sum(n[1:])
        # print(n)
        return np.abs(n[0,0])

    dgr = np.copy(domain_grads)
    number_of_domains, dim = dgr.shape

    #
    dgr /= np.sqrt(np.sum(dgr**2, axis=1))[:, np.newaxis]
    
    #
    g = np.random.randn(dim)
    g /= np.sqrt(np.sum(g**2))
    u_ = np.zeros(number_of_domains)
    
    iter = 0
    while not np.min(dgr@g/dim) >= epsilon:

        log()
        
        u_ = u_ + alpha*((1.+eff)*epsilon - (dgr@g)/dim)
        g = (1./number_of_domains)*np.sum(((1+u_).reshape(number_of_domains, 1))*dgr, axis=0)

        iter += 1
    
    log()
    return scale()*g

### Method 2 Version 0

In [ ]:
def gradient_maker(grads, rho=10.):
    log = lambda : print(f"Iter={iter}, Condition={np.min(G.T@G@s)}")
    norm = np.linalg.norm

    G = grads.T
    A = G.T@G
    g_ = np.mean(grads, axis=0)


    n, d = grads.shape

    #
    iter = 1

    s = np.random.randn(n)
    s /= np.sum(s).item()
    u = np.zeros(n)
    v = np.zeros(1)

    while np.min(G.T@G@s) < 0:

        # computing the gradient of the Lagrangian w.r.t s, u, v
        L_s = n*A@s - n*G.T@g_ + np.sum(-(u+rho*np.maximum(np.zeros(n), -A@s))*A*(A@s<=0), axis=0).T \
                                            + (v+rho*(np.sum(s)-1.))*np.ones(n)
        L_u = np.maximum(np.zeros(n), -A@s)
        L_v = np.sum(s)-1.

        #
        beta = 1 / iter
        alpha = beta/np.sqrt(norm(L_s)**2+norm(L_u)**2+norm(L_v)**2)

        # update rules
        s = s - alpha*L_s
        u = u + alpha*L_u
        v = v + alpha*L_v

        #
        log()
        iter += 1
    
    return G@s

### Method 2 Version 1: using `CVXPY`

In [ ]:
!pip install cvxpy

In [ ]:
def gradient_maker(grads):
    """
    - make sure to install `cvxpy`. you can use: `pip install cvxpy`
    - `grads` in a numpy's `ndarray`
    - `grads.shape == (n, d)`, where `n` is the number of domains and `d` is the dimension
    - this method will return a tuple of size two, where:
        * the first one is the generalized vector to use with size `d`
        * the second one is the weight vector of the linear combination
    - finally, use g, _ = gradient_maker(grads), if you have no need to use the 2nd return
    """

    import cvxpy as cp
    from numpy import linalg as la

    def nearestPD(A):

        B = (A + A.T) / 2
        _, s, V = la.svd(B)

        H = np.dot(V.T, np.dot(np.diag(s), V))

        A2 = (B + H) / 2

        A3 = (A2 + A2.T) / 2

        if isPD(A3):
            return A3

        spacing = np.spacing(la.norm(A))
        
        I = np.eye(A.shape[0])
        k = 1
        while not isPD(A3):
            mineig = np.min(np.real(la.eigvals(A3)))
            A3 += I * (-mineig * k**2 + spacing)
            k += 1

        return A3


    def isPD(B):
        try:
            _ = la.cholesky(B)
            return True
        except la.LinAlgError:
            return False
    
    #
    n, d = grads.shape
    G = grads.T
    g_ = np.mean(grads, axis=0).reshape(-1, 1)

    #
    P = nearestPD(n*G.T@G)
    q = -n*G.T@g_
    F = -G.T@G
    h = np.zeros(n, dtype=np.float32)
    A = np.ones(n, dtype=np.float32).reshape(1, -1)
    b = np.ones((1, 1), dtype=np.float32)

    # define opt variable
    x = cp.Variable(n)
    prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x, P) + q.T @ x),
                    [F @ x <= h,
                    A @ x == b])
    #
    prob.solve()
    s = np.array(x.value)

    return G@s, s

### Method 2 Version 3: Searching among different solvers and using scaling

In [ ]:
def gradient_maker(grads, solver=None):
    """
    - make sure to install `cvxpy`. you can use: `pip install cvxpy`
    - `grads` in a numpy's `ndarray`
    - `grads.shape == (n, d)`, where `n` is the number of domains and `d` is the dimension
    - this method will return a tuple of size two, where:
        * the first one is the generalized vector to use with size `d`
        * the second one is the weight vector of the linear combination
    - finally, use g, _ = gradient_maker(grads), if you have no need to use the 2nd return
    """

    import cvxpy as cp
    from numpy import linalg as la

    def nearestPD(A):

        B = (A + A.T) / 2
        _, s, V = la.svd(B)

        H = np.dot(V.T, np.dot(np.diag(s), V))

        A2 = (B + H) / 2

        A3 = (A2 + A2.T) / 2

        if isPD(A3):
            return A3

        spacing = np.spacing(la.norm(A))
        
        I = np.eye(A.shape[0])
        k = 1
        while not isPD(A3):
            mineig = np.min(np.real(la.eigvals(A3)))
            A3 += I * (-mineig * k**2 + spacing)
            k += 1

        return A3


    def isPD(B):
        try:
            _ = la.cholesky(B)
            return True
        except la.LinAlgError:
            return False

    #
    G = grads.T
    n, d = grads.shape
    g_ = np.mean(grads, axis=0).reshape(-1, 1)

    #
    P = nearestPD(n*G.T@G)
    q = -n*G.T@g_
    F = -G.T@G
    h = np.zeros(n, dtype=np.float32)
    A = np.ones(n, dtype=np.float32).reshape(1, -1)
    b = np.ones((1, 1), dtype=np.float32)

    # define opt variable
    x = cp.Variable(n)
    prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x, P) + q.T @ x),
                    [F @ x <= h,
                    A @ x == b])
    #
    if solver is None:
        return cp.OSQP
    prob.solve(solver=solver, verbose=False)
    s = np.array(x.value)

    return G@s, s

def gm_search(grads, return_first=True, verbose=0):
    import gc
    import cvxpy as cp
    from numpy.linalg import norm

    def gc_collect(*vars):
        for var in vars:
            del var
        gc.collect()
    
    def get_solver(solver):
        return eval('cp.'+solver)
    
    def gm_search_in_scales(grads, solver=None):
        min_inner = -float('inf')
        g = None
        scales = range(20)
        for scale in scales:
            try:
                _scale = 1e1**float(scale)
                dgr = np.copy(grads)
                dgr /= norm(dgr, axis=1).reshape(-1, 1)
                dgr *= _scale

                g_scaled, _ = gradient_maker(dgr, solver)

                # we need to scale it back
                g_scaled_back = np.mean(norm(grads, axis=1))/norm(g_scaled) * g_scaled

                # then, we will replace the scaled back g with g if min_inner is increased
                if np.min(grads@g_scaled_back) >= min_inner:
                    g = np.copy(g_scaled_back)
                    min_inner = np.min(grads@g_scaled_back)
                
                #
                if verbose==2:
                    print("----------------")
                    print(f"the minimum inner product found with scaling {_scale} is {np.min(dgr@g_scaled)}")
                    print(f"the minimum inner product found without scaling is {np.min(grads@g_scaled_back)}")

                #
                gc_collect(_scale, dgr, g_scaled, g_scaled_back)
            except:
                break
        return g
    
    # excluding: 'SCIP'
    qp_solvers = ['OSQP','CPLEX','NAG','ECOS','GUROBI','MOSEK','CVXOPT','SCS','XPRESS']
    
    #
    min_inner = -float('inf')
    g = None

    for solver in qp_solvers:

        if verbose>=1:
            print()
            print(f"############## Solver {solver} ##############")

        _g = gm_search_in_scales(grads, get_solver(solver))

        if _g is None:
            continue
        
        if verbose>=1:
            print()
            print(f"best result with solver = {solver} is {np.min(grads@_g)}")

        if return_first and np.min(grads@_g) > 0. :
            g = np.copy(_g)
            break

        if np.min(grads@_g) >= min_inner:
            g = np.copy(_g)
            min_inner = np.min(grads@_g)
        
        #
        gc_collect(_g)
    
    if g is None:
        print("FOUND NOTHING!!!!")
        g = np.mean(grads, axis=0)

    if verbose>=1:
        print()
        print("============================================================")
        print(f"final result is {np.min(grads@g)}")

    return g

### Method 3: Perceptron-based algorithm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

import torch
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm

In [ ]:
def gmaker(grads, epochs = 100, method="simple", start="zero", verbose=1):
    lr = 0.01
    n, d = grads.shape
    w = np.random.randn(d) if start=="normal" else np.zeros(d)
    for e in range(epochs):
        ge = grads[e%n]
        w_new = w + lr * (1. - (ge@w > 0)) * ge
        w = w_new if method=="simple" else w_new/np.linalg.norm(w_new)
    pos_count = np.sum(grads@w>0)
    total_count = n
    if verbose > 0:
        print(f"For {epochs} epochs: {pos_count} out of {n}")
    return w, (pos_count, max_count, total_count)

def gmaker_torch(grads, epochs = 100, method="simple", start="zero", verbose=1, device=None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') if device is None else device
    lr = 0.01
    n, d = grads.size()
    w = torch.randn(d) if start=="normal" else torch.zeros(d)
    w = w.to(device)
    for e in range(epochs):
        ge = grads[e%n]
        w_new = w + lr * (1. - (ge@w > 0).int()) * ge
        w = w_new if method=="simple" else w_new/torch.norm(w_new)
    pos_count = torch.sum(grads@w > 0)
    total_count = n
    if verbose > 0:
        print(f"For {epochs} epochs: {pos_count} out of {n}")
    return w, (pos_count, max_count, total_count)

#### Without `max_count` in the `return` statement

In [ ]:
def gradient_maker(grads, epochs = 100000, method="simple", start="zero", verbose=1, device=None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') if device is None else device
    lr = 0.01
    n, d = grads.size()
    w = torch.randn(d) if start=="normal" else torch.zeros(d)
    w = w.to(device)
    for e in range(epochs):
        ge = grads[e%n]
        w_new = w + lr * (1. - (ge@w > 0).int()) * ge
        w = w_new if method=="simple" else w_new/torch.norm(w_new)
    pos_count = torch.sum(grads@w > 0)
    total_count = n
    if verbose > 0:
        print(f"For {epochs} epochs: {pos_count} out of {n}")
    return w, (pos_count, total_count)